<a href="https://colab.research.google.com/github/FalseFeint/MertsMinions/blob/main/MRI_Segmentation_U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:



#                                 _     ___                        
#    __    __ _   _ __    ___    | |   / __|  __ _    ___    ___   
#   / _|  / _` | | '  \  / -_)   | |  | (__  / _` |  (_-<   / -_)  
#   \__|_ \__,_| |_|_|_| \___|  _|_|_  \___| \__,_|  /__/_  \___|  
# _|"""""_|"""""_|"""""_|"""""_|"""""_|"""""_|"""""_|"""""_|"""""| 
# "`-0-0-"`-0-0-"`-0-0-"`-0-0-"`-0-0-"`-0-0-"`-0-0-"`-0-0-"`-0-0-' 

"""
** camelCase!!!!!! **

i = int
dbl = double
flt = float
ten = tensor
arr = array
iArr = int array
char = character
str = string
dict = dictionary
lst = list
mat = Matrix
im = Image
"""
#   _____                              _____        _                 _   
#  |  __ \                            |  __ \      | |               | |  
#  | |__) _ __ ___   ___ ___ ___ ___  | |  | | __ _| |_ __ _ ___  ___| |_ 
#  |  ___| '__/ _ \ / __/ _ / __/ __| | |  | |/ _` | __/ _` / __|/ _ | __|
#  | |   | | | (_) | (_|  __\__ \__ \ | |__| | (_| | || (_| \__ |  __| |_ 
#  |_|   |_|  \___/ \___\___|___|___/ |_____/ \__,_|\__\__,_|___/\___|\__|

#  _____        _                                                   _        _   _             
#  |  __ \      | |            /\                                   | |      | | (_)            
#  | |  | | __ _| |_ __ _     /  \  _   _  __ _ _ __ ___   ___ _ __ | |_ __ _| |_ _  ___  _ __  
#  | |  | |/ _` | __/ _` |   / /\ \| | | |/ _` | '_ ` _ \ / _ \ '_ \| __/ _` | __| |/ _ \| '_ \ 
#  | |__| | (_| | || (_| |  / ____ \ |_| | (_| | | | | | |  __/ | | | || (_| | |_| | (_) | | | |
#  |_____/ \__,_|\__\__,_| /_/    \_\__,_|\__, |_| |_| |_|\___|_| |_|\__\__,_|\__|_|\___/|_| |_|
#                                         __/ |                                                
#                                        |___/                                                 

#   _______        _       _             
#  |__   __|      (_)     (_)            
#     | |_ __ __ _ _ _ __  _ _ __   __ _ 
#     | | '__/ _` | | '_ \| | '_ \ / _` |
#     | | | | (_| | | | | | | | | | (_| |
#     |_|_|  \__,_|_|_| |_|_|_| |_|\__, |
#                                   __/ |
#                                  |___/ 

"""
TA: Can we follow an example closely (especially for baseline given that we 
are noobs)
What should we include in the report?
What are your thoughts on skullstripping?
What are ways we can be innovative with our implementation?
Any advice? Anything we should watch out for?
How can we use the GPU in google colab?

Professor said this:
the expectation is that you will describe your problem, data and code that performs I/O and pre-processing.
You can also offer a visual exploration of the data and show some results with simple baseline method,
You can include preliminary results with implemented ML algorithm.

The main text should NOT exceed 4 pages.

"""

In [2]:
import numpy as np
import torch
import torch.nn as nn

#UNet
Create UNet architecture

In [6]:
#   _    _ _   _      _   
#  | |  | | \ | |    | |  
#  | |  | |  \| | ___| |_ 
#  | |  | | . ` |/ _ | __|
#  | |__| | |\  |  __| |_ 
#   \____/|_| \_|\___|\__|

class UNet(nn.Module): 
  def __init__(self, iInChannel=3, iOutChannel=1, iInFeatures=32):
    super(UNet, self).__init__()

    # TODO: See if we can reduce this to for-loop using eval
    # TODO: Look into initializing all pool layers at once
    iFeatures = iInFeatures

    # Declare all our down blocks and max pool layers
    self.downBlock1 = UNet._block(iInChannels, iFeatures, name="downBlock1")
    self.maxPool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.downBlock2 = UNet._block(iFeatures, iFeatures * 2, name="downBlock2")
    self.maxPool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.downBlock3 = UNet._block(iFeatures * 2, iFeatures * 4, name="downBlock3")
    self.maxPool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.downBlock4 = UNet._block(iFeatures * 4, iFeatures * 8, name="downBlock4")
    self.maxPool4 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Declare bottom of UNet
    self.bottleneck = UNet._block(iFeatures * 8, features * 16, name="bottleneck")

    # TODO: See if we can reduce this to for-loop using eval
    # TODO: Look into initializing all pool layers at once
    # Declare up-blocks and up-convolutions
    self.upconv4 = nn.ConvTranspose2d(
        iFeatures * 16, iFeatures * 8, kernel_size=2, stride=2)
    self.upBlock4 = UNet._block((iFeatures * 8) * 2, iFeatures * 8, name="upBlock4")
    
    self.upconv3 = nn.ConvTranspose2d(
        iFeatures * 8, iFeatures * 4, kernel_size=2, stride=2)
    self.upBlock3 = UNet._block((iFeatures * 4) * 2, iFeatures * 4, name="upBlock3")
    
    self.upconv2 = nn.ConvTranspose2d(
        iFeatures * 4, iFeatures * 2, kernel_size=2, stride=2)
    self.upBlock2 = UNet._block((iFeatures * 2) * 2, iFeatures * 2, name="upBlock2")
    
    self.upconv1 = nn.ConvTranspose2d(
        iFeatures * 2, iFeatures, kernel_size=2, stride=2)
    self.upBlock1 = UNet._block(iFeatures * 2, iFeatures, name="upBlock1")

    self.conv = nn.Conv2d(
        in_channels=features, out_channels=out_channels, kernel_size=1
    )

    # 
    def forward(self, x):
        # Connect down blocks with max-pool layers
        downBlock1 = self.downBlock1(x)
        downBlock2 = self.downBlock2(self.maxPool1(downBlock1))
        downBlock3 = self.downBlock3(self.maxPool2(downBlock2))
        downBlock4 = self.downBlock4(self.maxPool3(downBlock3))

        # Add bottleneck
        bottleneck = self.bottleneck(self.pool4(downBlock4))

        # Connect up blocks with up convolutions and skip connections
        upBlock4 = self.upconv4(bottleneck)
        upBlock4 = torch.cat((upBlock4, downBlock4), dim=1)
        upBlock4 = self.upBlock4(upBlock4)
        upBlock3 = self.upconv3(upBlock4)
        upBlock3 = torch.cat((upBlock3, downBlock3), dim=1)
        upBlock3 = self.upBlock3(upBlock3)
        upBlock2 = self.upconv2(upBlock3)
        upBlock2 = torch.cat((upBlock2, downBlock2), dim=1)
        upBlock2 = self.upBlock2(upBlock2)
        upBlock1 = self.upconv1(upBlock2)
        upBlock1 = torch.cat((upBlock1, downBlock1), dim=1)
        upBlock1 = self.upBlock1(upBlock1)
        return torch.sigmoid(self.conv(upBlock1))


    """ 
    This function is useful for blocking together convolutions, activations, 
    and max pooling for each "meta-layer" in our U-Net architecture 

    Convolution layers in this block have specified in channels, 
    features, a kernel size of 3, padding of 1 and no bias

    We use batch normalization to reduce internal covariance

    ReLU is used for non-linearity activation
    """
    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )
